In [ ]:
import pandas as pd
import io
import evaluate
import numpy as np
import os
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer, set_seed

device = "cuda:0" if torch.cuda.is_available() else "cpu"

def get_data(train_path, test_path, random_seed, test_size=0.2, subset_frac=1.0):
    with open(train_path, 'r') as file:
        json_data = io.StringIO(file.read())
        train_df = pd.read_json(json_data, lines=True)
    with open(test_path, 'r') as file:
        json_data = io.StringIO(file.read())
        test_df = pd.read_json(json_data, lines=True)

    # Sample a fraction of training data (for faster testing)
    if subset_frac < 1.0:
        train_df = train_df.sample(frac=subset_frac, random_state=random_seed)

    train_df, val_df = train_test_split(train_df, test_size=test_size, stratify=train_df['label'], random_state=random_seed)
    return train_df, val_df, test_df

def preprocess_function(examples, **fn_kwargs):
    return fn_kwargs['tokenizer'](examples["text"], truncation=True, max_length=512)

def compute_metrics(eval_pred):
    f1_metric = evaluate.load("f1")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    results = f1_metric.compute(predictions=predictions, references=labels, average="micro")
    return results

if __name__ == '__main__':
    train_path = '/content/subtaskA_train_monolingual.jsonl'
    test_path = '/content/subtaskA_monolingual.jsonl'
    random_seed = 1011
    subset_frac = 0.05  # Train on 5% of the dataset

    llm_model = "prajjwal1/bert-tiny"

    train_df, valid_df, test_df = get_data(train_path, test_path, random_seed, test_size=0.2, subset_frac=subset_frac)

    id2label = {0: "human", 1: "machine"}
    label2id = {"human": 0, "machine": 1}

    train_dataset = Dataset.from_pandas(train_df)
    valid_dataset = Dataset.from_pandas(valid_df)

    tokenizer = AutoTokenizer.from_pretrained(llm_model)
    tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True, fn_kwargs={'tokenizer': tokenizer})
    tokenized_valid_dataset = valid_dataset.map(preprocess_function, batched=True, fn_kwargs={'tokenizer': tokenizer})
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    model = AutoModelForSequenceClassification.from_pretrained(
        llm_model, num_labels=2, id2label=id2label, label2id=label2id
    ).to(device)

    training_args = TrainingArguments(
        output_dir='./bert',
        learning_rate=2e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,  # Quick testing
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        logging_steps=100
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_valid_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    best_model_path = './bert/best/'
    os.makedirs(best_model_path, exist_ok=True)
    trainer.save_model(best_model_path)

    # Load best model and predict on test set
    best_model = AutoModelForSequenceClassification.from_pretrained(
        best_model_path, num_labels=2, id2label=id2label, label2id=label2id
    ).to(device)

    test_dataset = Dataset.from_pandas(test_df)
    tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True, fn_kwargs={'tokenizer': tokenizer})

    trainer = Trainer(
        model=best_model,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    predictions = trainer.predict(tokenized_test_dataset)
    preds = np.argmax(predictions.predictions, axis=-1)

    predictions_df = pd.DataFrame({'id': test_df['id'], 'label': preds})
    os.makedirs('./output/bert/', exist_ok=True)
    predictions_df.to_json('./output/bert/subtaskA_monolingual.jsonl', orient='records', lines=True)


config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/4790 [00:00<?, ? examples/s]

Map:   0%|          | 0/1198 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-2-8d9abf7b646c>:76: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Map:   0%|          | 0/34272 [00:00<?, ? examples/s]

<ipython-input-2-8d9abf7b646c>:100: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
